# More Practice With SQL Queries - Lab

## Introduction

In this lesson, we'll run through some practice questions to refresh our knowledge of SQL Queries!

## Objectives

You will be able to:

- Practice your SQL knowledge

## Getting Started

As in previous labs, we'll make use of the `sqlite3` library as well as `pandas`. By combining them, we'll be able to write our queries as python strings, and make sure that the results are always returned as a pandas DataFrame. 

We'll start by loading both libraries and connecting to the database we'll be using for this lab, `data.sqlite`. You may remember this database from a previous lab. As a refresher, here's the ERD diagram for this database: 

<img src='images/Database-Schema.png'>

In the cell below:

* Import the necessary libraries `pandas` and `sqlite3`
* Establish a connection to the database `data.sqlite`
* Get the `cursor` from the connection and store it in the variable `c`.

In [1]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('data.sqlite')
c = conn.cursor()

## Basic Queries

Now, let's review basic SQL queries. In the cell below:

* Write a query that gets the first name, last name, phone number, address, and credit limit for all customers in California with a credit limit greater than 25000.00. 

In [12]:
# For the first query, the boilerplate for getting 
#the query into a dataframe has been provided for you
c.execute("""SELECT contactFirstName, contactLastName, phone, addressLine1 FROM customers
                    WHERE (state = "CA") & (creditLimit > 25000.00);""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,contactFirstName,contactLastName,phone,addressLine1
0,Susan,Nelson,4155551450,5677 Strong St.
1,Julie,Murphy,6505555787,5557 North Pendale Street
2,Juri,Hashimoto,6505556809,9408 Furth Circle
3,Julie,Young,6265557265,78934 Hillside Dr.
4,Valarie,Thompson,7605558146,361 Furth Circle
5,Julie,Brown,6505551386,7734 Strong St.
6,Brian,Chandler,2155554369,6047 Douglas Av.
7,Sue,Frick,4085553659,3086 Ingle Ln.
8,Steve,Thompson,3105553722,3675 Furth Circle
9,Sue,Taylor,4155554312,2793 Furth Circle


In [5]:
c.description

(('contactFirstName', None, None, None, None, None, None),
 ('contactLastName', None, None, None, None, None, None),
 ('phone', None, None, None, None, None, None),
 ('addressLine1', None, None, None, None, None, None))

In [7]:
c.execute("""SELECT * FROM customers""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",,Nantes,,44000,France,1370,21000.0
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,,Las Vegas,NV,83030,USA,1166,71800.0
2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,1611,117300.0
3,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",,Nantes,,44000,France,1370,118200.0
4,121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,,Stavern,,4110,Norway,1504,81700.0


In [9]:
c.execute('pragma table_info(customers)').fetchall()

[(0, 'customerNumber', 'INTEGER', 0, None, 1),
 (1, 'customerName', 'TEXT', 0, None, 0),
 (2, 'contactLastName', 'TEXT', 0, None, 0),
 (3, 'contactFirstName', 'TEXT', 0, None, 0),
 (4, 'phone', 'TEXT', 0, None, 0),
 (5, 'addressLine1', 'TEXT', 0, None, 0),
 (6, 'addressLine2', 'TEXT', 0, None, 0),
 (7, 'city', 'TEXT', 0, None, 0),
 (8, 'state', 'TEXT', 0, None, 0),
 (9, 'postalCode', 'INTEGER', 0, None, 0),
 (10, 'country', 'TEXT', 0, None, 0),
 (11, 'salesRepEmployeeNumber', 'INTEGER', 0, None, 0),
 (12, 'creditLimit', 'REAL', 0, None, 0)]

#### Expected Output

<img src='images/expected-output-1.png'>

## Aggregate Functions and GROUP BY

Next, write a query that get sthe average credit limit per state.

In [13]:
c.execute("""SELECT state, AVG(creditLimit) FROM customers
                    GROUP BY state""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,state,AVG(creditLimit)
0,,61839.726027
1,BC,89950.000000
2,CA,83854.545455
3,CT,57350.000000
4,Co. Cork,0.000000


#### Expected Output

<img src='images/expected-output-2.png'>

## JOINs

Now, write a query that uses JOIN statements to get the customer name, customer number, order number, status, and quantity ordered. Print only the head of this DataFrame. 

In [14]:
c.execute("""SELECT customerName, customerNumber, orderNumber, status, quantityOrdered FROM customers
                    JOIN orders USING (customerNumber)
                    JOIN orderdetails USING (orderNumber);""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,customerName,customerNumber,orderNumber,status,quantityOrdered
0,Online Diecast Creations Co.,363,10100,Shipped,22
1,Online Diecast Creations Co.,363,10100,Shipped,30
2,Online Diecast Creations Co.,363,10100,Shipped,49
3,Online Diecast Creations Co.,363,10100,Shipped,50
4,"Blauer See Auto, Co.",128,10101,Shipped,25


#### Expected Output

<img src='images/joins.png'>

## HAVING and ORDER BY

Now, return the customerName, customrerNumber, productName, productCode and total number ordered for any product a customer has bought 10 or more of cumulatively. Sort the rows in descending order by the quantity ordered. 

**_Hint_**: For this one, you'll need to make use of HAVING, GROUP BY, and ORDER BY--make sure you get the order of them correct!

In [25]:
c.execute("""SELECT customerName, customerNumber, productName, productCode, SUM(quantityOrdered) FROM customers
                    JOIN orders USING (customerNumber)
                    JOIN orderdetails USING (orderNumber)
                    JOIN products USING (productCode)
                    GROUP BY 4
                    HAVING SUM(quantityOrdered) >= 10
                    ORDER BY SUM(quantityOrdered) DESC;""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()
df.loc[(df.customerName == "Euro+ Shopping Channel") & (df.productCode )]

,customerName,customerNumber,productName,productCode,SUM(quantityOrdered)
6,Euro+ Shopping Channel,141,1969 Harley Davidson Ultimate Chopper,S10_1678,1057
14,Euro+ Shopping Channel,141,1997 BMW R 1100 S,S24_1578,1033
17,Euro+ Shopping Channel,141,2002 Suzuki XREO,S12_2823,1028
25,Euro+ Shopping Channel,141,1996 Moto Guzzi 1100i,S10_2016,999
26,Euro+ Shopping Channel,141,1940 Ford Pickup Truck,S18_1097,999
28,Euro+ Shopping Channel,141,1982 Camaro Z28,S700_2824,997
29,Euro+ Shopping Channel,141,1939 Cadillac Limousine,S18_4668,995
35,Euro+ Shopping Channel,141,1996 Peterbilt 379 Stake Bed with Outrigger,S32_3522,988
37,Euro+ Shopping Channel,141,2003 Harley-Davidson Eagle Drag Bike,S10_4698,985
46,Euro+ Shopping Channel,141,1958 Setra Bus,S12_1666,972


#### Expected Output

<img src='images/having_order.png'>

## Subqueries

Finally, get the first name, last name, employee number, and office code for employees from an office with less than 5 employees. 

#### Expected Output

<img src='images/expected-output-5.png'>

# Summary

In this lesson, we reviewed all the major concepts and keywords associated with SQL queries!